In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
sys.path.insert(0, "../src/")
import re
import numpy as np
import pandas as pd
from tqdm import tqdm
from handcrafted_features import DocBasedFeatureExtractor, Doc2VecChunkVectorizer, CorpusBasedFeatureExtractor
from utils import get_doc_paths, read_labels
import pickle

raw_docs_dir = "../data/raw_docs/"
labels_dir = "../data/labels/"
extracted_features_dir = "../data/extracted_features/"

lang = "eng" #"ger"
doc_paths = get_doc_paths(raw_docs_dir, lang)

sentences_per_chunk = 200
#d2vcv = Doc2VecChunkVectorizer(lang, sentences_per_chunk)
#d2vcv.fit_transform(reduced_doc_paths)

In [ ]:
# file_name_mapper = {'eng':['Blackmore_R-D_Lorna-Doone_1869',
#                     'Bulwer-Lytton_Edward_Paul-Clifford_1830',
#                     'Conrad_Joseph_The-Secret-Sharer_1910',
#                     'Parsons_Eliza_The-Castle-of-Wolfenbach_1793',
#                     'Richardson_Samuel_Sir-Charles-Grandison_1753',
#                     'Austen_Jane_Northanger-Abbey_1818',
#                     'Cleland_John_Fanny-Hill_1748',
#                     'Defoe_Daniel_Roxana_1724',
#                     'Fielding_Henry_Amelia_1752',
#                     'Kingsley_Charles_The-Water-Babies_1863',
#                     'Le-Queux_William_The-Invasion-of-1910_1906',
#                     'Surtees_Robert_Jorrocks-Jaunts-and-Jollities_1831'],
#                     'ger': ['Eichendorff_Joseph_Auch-ich-war-in-Arkadien_1832',
#                     'Eichendorff_Joseph_Die-Gluecksritter_1841',
#                     'Eichendorff_Joseph_Libertas-und-ihrer-Freier_1848',
#                     'Eichendorff_Joseph_Viel-Laermen-um-Nichts_1832',
#                     'Goethe_Johann-Wolfgang_Unterhaltungen-deutscher-Ausgewanderten_1795',
#                     'Zschokke_Johann_Addrich-im-Moos_1825',
#                     'Zschokke_Johann_Der-Freihof-von-Aarau_1823',
#                     'Zschokke_Johann_Die-Rose-von-Disentis_1844']}
# for lang in ["eng", "ger"]:
#     file_names = file_name_mapper[lang]
#     print(file_names)
#     doc_paths = ['../data/raw_docs/' + lang + '/' + filename + '.txt' for filename in file_names]
#     print(doc_paths)

    #d2vcv = Doc2VecChunkVectorizer(lang, sentences_per_chunk)
    #d2vcv.fit_transform(reduced_doc_paths)

all_chunk_based_features = []
all_book_based_features = []
all_average_sbert_sentence_embeddings = []
all_doc2vec_chunk_embeddings = []
for index, doc_path in enumerate(doc_paths):
    #if there is an error with chunk_index, rerun the Doc2VecChunkVectorizer for the book where the error occurs
    fe = DocBasedFeatureExtractor(lang, doc_path, sentences_per_chunk)
    chunk_based_features, book_based_features, average_sbert_sentence_embeddings, doc2vec_chunk_embeddings = fe.get_all_features()
    all_chunk_based_features.extend(chunk_based_features)
    all_book_based_features.append(book_based_features)
    all_average_sbert_sentence_embeddings.append(average_sbert_sentence_embeddings)
    all_doc2vec_chunk_embeddings.append(doc2vec_chunk_embeddings)
    if index > 0 and index % 10 == 0:
        feature_extraction_path = '/home/annina/scripts/great_unread_nlp/data/feature_extraction/'
        f = open(feature_extraction_path + lang+ '/features_' + str(index) + '.pkl', 'wb')  
        pickle.dump([all_chunk_based_features, all_book_based_features, all_average_sbert_sentence_embeddings, all_doc2vec_chunk_embeddings], f, -1)
        f.close()
        print('pickle', index)


pickle 10
pickle 20


/home/annina/anaconda3/envs/nlp/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3419: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/annina/anaconda3/envs/nlp/lib/python3.9/site-packages/numpy/core/_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


pickle 30
pickle 40
pickle 50
pickle 60
pickle 70
pickle 80
pickle 90
pickle 100
pickle 110
pickle 120
pickle 130
pickle 140
pickle 150
pickle 160
pickle 170
pickle 180
pickle 190
pickle 200
pickle 210
pickle 220
pickle 230
pickle 240
pickle 250
pickle 260
pickle 270
pickle 280
pickle 290
pickle 300
pickle 310
pickle 320
pickle 330
pickle 340
pickle 350
pickle 360
pickle 370
pickle 380
pickle 390
pickle 400
pickle 410
pickle 420
pickle 430
pickle 440
pickle 450
pickle 460
pickle 470
pickle 480
pickle 490
pickle 500
pickle 510
pickle 520
pickle 530
pickle 540
pickle 550
pickle 560


In [ ]:
cbfe = CorpusBasedFeatureExtractor(lang, doc_paths, all_average_sbert_sentence_embeddings, all_doc2vec_chunk_embeddings)
all_corpus_based_features = cbfe.get_all_features()


import os
import pandas as pd

book_df = pd.DataFrame(all_book_based_features)
book_df = book_df.merge(all_corpus_based_features, on="book_name")
book_and_averaged_chunk_df = book_df.merge(pd.DataFrame(all_chunk_based_features).groupby("book_name").mean().reset_index(drop=False), on="book_name")

chunk_df = pd.DataFrame(all_chunk_based_features)
chunk_and_copied_book_df = chunk_df.merge(pd.DataFrame(all_book_based_features), on="book_name")
chunk_and_copied_book_df = chunk_and_copied_book_df.merge(all_corpus_based_features, on="book_name")

os.makedirs(f"{extracted_features_dir}/{lang}", exist_ok=True)
book_df.to_csv(f"{extracted_features_dir}/{lang}/book_df.csv", index=False)
book_and_averaged_chunk_df.to_csv(f"{extracted_features_dir}/{lang}/book_and_averaged_chunk_df.csv", index=False)
chunk_df.to_csv(f"{extracted_features_dir}/{lang}/chunk_df.csv", index=False)
chunk_and_copied_book_df.to_csv(f"{extracted_features_dir}/{lang}/chunk_and_copied_book_df.csv", index=False)